# Coastal Erosion Simulation: Finding the Initial Parameters

## Introduction
This notebook tries to find reasonable estimates for parameters modeling the simulation world of the Norfolk coastline.

The simulation incorporates data from NOAA buoy **62144**, alongside empirical equations derived from relevant literature, to calculate wave and coastal parameters such as wave height, wave speed, sand pull, ground pull, and others. Below is an explanation of the steps, models, and equations used in this analysis.

#### Sources:

https://www.ndbc.noaa.gov/
Environment Agency, Sea State Report: Felixstowe Wave Buoy (2013): https://www.coastalmonitoring.org/pdf_download/?metadata_id=486318

Bouws, E., & Komen, G. J. (1983). On the balance between growth and dissipation in an extreme depth-limited wind-sea in the southern North Sea. Journal of physical oceanography, 13(9), 1653-1658.

Walkden, M. J. A., & Hall, J. W. (2005). A predictive mesoscale model of the erosion and profile development of soft rock shores. Coastal Engineering, 52(6), 535-563.

Bruun, P. (1962). Sea-level rise as a cause of shore erosion. Journal of the Waterways and Harbors division, 88(1), 117-130.

## 1. Fetching Buoy Data
The simulation starts by fetching real-time wave data from NOAA's database for the buoy nearest Norfolk. Key parameters such as **wave height** and **wave speed** are extracted and averaged.

In [5]:
import pandas as pd

# Fetch real-time data for the closest buoy
buoy_id = 62144
data_url = f"https://www.ndbc.noaa.gov/data/realtime2/{buoy_id}.txt"

# Load buoy data
try:
    data = pd.read_csv(data_url, delim_whitespace=True, skiprows=[1], na_values=["MM"])
    print(data.head())
except Exception as e:
    print(f"Error fetching data for buoy {buoy_id}: {e}")

C:\Users\sonia\AppData\Local\Temp\ipykernel_2524\896513729.py:9: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  data = pd.read_csv(data_url, delim_whitespace=True, skiprows=[1], na_values=["MM"])


    #YY  MM  DD  hh  mm  WDIR  WSPD  GST  WVHT  DPD  APD  MWD    PRES  ATMP  \
0  2025   1  19  13   0   180   3.6  NaN   0.7  NaN  NaN  NaN  1020.9   2.6   
1  2025   1  19  12   0   190   5.1  NaN   0.5  NaN  NaN  NaN  1021.3   2.4   
2  2025   1  19  11   0   190   5.7  NaN   0.7  NaN  NaN  NaN  1021.9   2.3   
3  2025   1  19  10   0   180   4.6  NaN   0.7  NaN  NaN  NaN  1022.1   1.8   
4  2025   1  19   9   0   190   5.7  NaN   0.7  NaN  NaN  NaN  1022.3   1.6   

   WTMP  DEWP  VIS  PTDY  TIDE  
0   NaN   0.3    5  -1.2   NaN  
1   NaN   0.2    5  -1.0   NaN  
2   NaN   0.3    2  -0.3   NaN  
3   NaN   0.1    5  -0.1   NaN  
4   NaN   0.1    5  -0.1   NaN  


Extract the relevant data

In [6]:
# Drop rows with missing wave height
data = data.dropna(subset=["WVHT"])

wave_height = data["WVHT"].mean()  # Significant wave height
wave_speed = data["WSPD"].mean()  

# Output parameters
print(f"WAVE_HEIGHT: {wave_height}") # m
print(f"WAVE_SPEED: {wave_speed}") # m/s

WAVE_HEIGHT: 0.572
WAVE_SPEED: 3.292


## 2. Estimating the Unknown

Using models from the literature, empirical findings, and some heavy approximations. (Details in the report)

In [11]:
import numpy as np

wave_height = 0.564 # (m), based on data from 18/01
wave_speed = 4.088 # (m/s), based on data from 18/01

# 1. Wave Frequency
T = 4.81  # representative wave period (s)
wave_freq = 1 / T

# 2. Wave Amplitude
wave_amplitude = wave_height / 2  # half of significant wave height

# 3. Wave Volume (calculated from wave height and wavelength)
g = 9.81  # gravity (m/s^2)
W = 1 # assumed wave width (m)
wavelength = (g * T**2) / (2 * np.pi)  # Wavelength using deep water wave formula
wave_volume = (wave_height / 2) * wavelength * 1  # Wave volume in m³/wave

# 4. Wave Spread
wave_spread = 12  # degrees, assumed

# 5. Wave Decay
wave_decay = 0.038 # Based on bottom friction in JONSWAP model

# 6. Wave Cutoff Depth
wave_cutoff = 0.23 * wave_height  # Cutoff depth based on H_s

# 7. Wave Retreat Coefficient
wave_retreat_coeff = 0.8  # estimated

# 8. Ground Pull (Erosion Rate Model)

# Constants
rho = 1025  # water density (kg/m^3)
R = 1.27e6  # material resistance (N/m^2) # assumed
f1 = 0.8  # wave breaking term (estimated)
f2 = 1  # slope constant, estimated
z = 0.05  # shore platform slope (radians), estimated
f3_t = 1  # time-dependent tidal influence, assumed
wa = 0.39 # empirical transport coefficient (CERC model)

F = rho * g * wave_height**2  # Wave force in N/m² (calculated from wave height)
ground_pull = (F / R) * f1 * f3_t * np.tan(f2 * z)  # Erosion rate in m/year 

# 9. Sand Pull (CERC Equation)
sand_pull = wa * wave_height**(5/2) * np.sin(2 * np.radians(wave_spread))  # Sand pull in m³/year (CERC model)

# Output results with updated precision
print(f"Wave Speed: {wave_speed:.6f} m/s")
print(f"Wave height: {wave_height:.6f} m")
print(f"Wave Frequency: {wave_freq:.6f} Hz")
print(f"Wave Amplitude: {wave_amplitude:.6f} m")
print(f"Wave Volume: {wave_volume:.6f} m³/wave")
print(f"Wave Spread: {wave_spread} degrees")

print(f"Wave Decay: {wave_decay}")
print(f"Wave Cutoff Depth: {wave_cutoff:.6f} m")
print(f"Wave Retreat Coefficient: {wave_retreat_coeff}")

print(f"Sand Pull: {sand_pull:.6f} m³/year")
print(f"Ground Pull: {ground_pull:.6f} m/year")

Wave Speed: 4.088000 m/s
Wave height: 0.564000 m
Wave Frequency: 0.207900 Hz
Wave Amplitude: 0.282000 m
Wave Volume: 10.186580 m³/wave
Wave Spread: 12 degrees
Wave Decay: 0.038
Wave Cutoff Depth: 0.129720 m
Wave Retreat Coefficient: 1.8
Sand Pull: 0.037894 m³/year
Ground Pull: 0.000101 m/year
